XGBoost Regression Project - Problem Statement

1.Problem statement
.this dataset comprises used cars sold on cardekho.com in india as well as important features of these cars.
.if user can predict the price of the car based on input features.
.Prediction results can be used to give new seller the price suggestion based on market condition.

2.Data Collection.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

# load data
df = pd.read_csv(r"./dataset/cardekho_dataset.csv", index_col=[0])

df.head()


,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


3.Data Cleaning
Handling Missing values , dublicates, check data type, understand dataset.

In [2]:
#check null value
# check features with nan value
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [3]:
# Remove Unneccessary columns
df.drop(columns=['car_name', 'brand'], inplace=True, errors='ignore')

df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [4]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [5]:
# Getting All Different Types of features
num_features = [feature for feature in df.columns if df[feature].dtype != 'o']
print("Num of Numerical Features :",len(num_features))

#categorial features
cat_features = [feature for feature in df.columns if df[feature].dtype == 'o']
print("Num of Categorical features :", len(cat_features))

# Discreate features
discrete_features = [feature for feature in num_features if len(df[feature].unique())<=25]
print("Num of Discrete Features :", len(discrete_features))

#continuous Features
continueus_feature = [feature for feature in num_features  if feature not in discrete_features]
print("Num of Continuous Features :", len(continueus_feature))


Num of Numerical Features : 11
Num of Categorical features : 0
Num of Discrete Features : 5
Num of Continuous Features : 6


In [6]:
## Independent and dependent features
from sklearn.model_selection import train_test_split
x = df.drop(['selling_price'], axis=1)
y = df['selling_price']

x.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


Feature Encoding and Scalling
one Hot Encoding for columns which had lesser unique values adn ordinal

.one hot encoding is a process by which categorical variables are converted into a from  that could be provided to ML algorithms to do a better job in predicton.

In [7]:
#len of model
len(df['model'].unique())

120

In [8]:
#value count
df['model'].value_counts()

model
i20            906
Swift Dzire    890
Swift          781
Alto           778
City           757
              ... 
Ghibli           1
Altroz           1
GTC4Lusso        1
Aura             1
Gurkha           1
Name: count, Length: 120, dtype: int64

In [9]:
#label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x['model'] = le.fit_transform(x['model'])

x.head()


,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [10]:
#seller_type
len(df['seller_type'].unique()),len(df['fuel_type'].unique()),len(df['transmission_type'].unique())

(3, 5, 2)

In [11]:
num_features = x.select_dtypes(include=['int64', 'float64']).columns
cat_features = x.select_dtypes(include=['object']).columns

onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('OneHot', oh_transformer, onehot_columns),
        ('Scaler', numeric_transformer, num_features)
    ],
    remainder='drop'
)

x_transformed = preprocessor.fit_transform(x)


import pandas as pd

x_transformed = pd.DataFrame(x_transformed)
print(x_transformed.head())



    0    1    2    3    4    5    6         7         8         9         10  \
0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.983562  1.247335 -0.000276 -1.324259   
1  1.0  0.0  0.0  0.0  0.0  1.0  1.0 -0.343933 -0.690016 -0.192071 -0.554718   
2  1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.647309  0.084924 -0.647583 -0.554718   
3  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.983562 -0.360667  0.292211 -0.936610   
4  0.0  0.0  1.0  0.0  0.0  0.0  1.0 -0.012060 -0.496281  0.735736  0.022918   

         11        12  
0 -1.263352 -0.403022  
1 -0.432571 -0.403022  
2 -0.479113 -0.403022  
3 -0.779312 -0.403022  
4 -0.046502 -0.403022  


In [12]:
# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(x_transformed, y, test_size=0.2 , random_state=42)
x_train.shape, x_test.shape

x_train 

,0,1,2,3,4,5,6,7,8,9,10,11,12
11210,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.319814,0.283541,-2.038093,1.753906,2.662498,-0.403022
1347,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.339555,-0.883751,0.992261,-0.550880,-0.386028,-0.403022
10363,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.339555,-0.961245,-0.168096,0.890331,3.274530,-0.403022
316,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.319814,0.143045,-0.455788,0.020999,0.388902,-0.403022
10638,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.315436,0.478051,0.157955,-0.554718,-0.504712,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.319814,1.397111,0.256249,-0.456846,-0.274327,2.073444
13418,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.647309,0.065551,-0.865749,0.214823,0.060778,-0.403022
5390,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.319814,-0.690016,0.193916,-0.936610,-0.780708,-0.403022
860,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.339555,-0.786884,-0.263994,-0.554718,-0.435829,-0.403022


MODEL Training And Model Selection

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [14]:
## Create a Function to Evaluate  Model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2 = r2_score(true, predicted)

    return mae, rmse, r2,

In [ ]:
## Beginning Model Training
models = {
    # "LinearRegression":LinearRegression(),
    # "Lasso":Lasso(),
    # "Ridge":Ridge(),
    "K-Neighbors Regressor":KNeighborsRegressor(),
    # "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),
    # "AdaboostRegression":AdaBoostRegressor()
    "XGBRegressor":XGBRegressor()
}

for name, model in models.items():

    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(name)
    print("---------Model performance for Training set-------------")
    # print(f"RMSE: {model_train_rmse:.4f}")
    # print(f"MAE : {model_train_mae:.4f}")
    print(f"R2  : {model_train_r2:.4f}")

    print("---------Model performance for Test set-------------")
    # print(f"RMSE: {model_test_rmse:.4f}")
    # print(f"MAE : {model_test_mae:.4f}")
    print(f"R2  : {model_test_r2:.4f}")
    print("="*40, "\n")


    

K-Neighbors Regressor
---------Model performance for Training set-------------
R2  : 0.8688
---------Model performance for Test set-------------
R2  : 0.8929

Decision Tree
---------Model performance for Training set-------------
R2  : 0.9995
---------Model performance for Test set-------------
R2  : 0.8605

Random Forest Regressor
---------Model performance for Training set-------------
R2  : 0.9778
---------Model performance for Test set-------------
R2  : 0.9248

XGBTRegressor
---------Model performance for Training set-------------
R2  : 0.9900
---------Model performance for Test set-------------
R2  : 0.8693



In [29]:
## Initialize few parameter for Hyperparamter tuning
knn_parms = {"n_neighbors":[2, 3, 10, 20, 40, 50]}
rf_params = {"max_depth":[5, 8, 15, None, 10],
             "max_features":[5, 7, "auto", 8],
             "min_samples_split":[2, 8, 15, 20],
             "n_estimators":[100 , 200, 500, 1000]
             }

xgboost_params = {
    "learning_rate": [0.1, 0.01],
    "max_depth": [5, 8, 12, 20],
    "n_estimators": [100, 200, 300],
    "colsample_bytree": [0.3, 0.5, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0],
    "gamma": [0, 0.1, 0.3],
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 2, 5]
}




In [30]:
#MOdels list for Hyperparameter tuning
randomcv_models = [('KNN', KNeighborsRegressor(), knn_parms),
                   ("RF",RandomForestRegressor(), rf_params),
                   ("xgboost",XGBRegressor(), xgboost_params),
                   ]

In [31]:
# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

model_param ={}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                param_distributions=params,
                                n_iter=20,
                                cv=3,
                                verbose=2,
                                n_jobs=-1,
                                random_state=42)
    random.fit(x_train, y_train)
    model_param[name] = random.best_params_

# print(model_param)
for model_name in model_param:
    print(f"------------------Best Params for {model_name}----------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
------------------Best Params for KNN----------------------
{'n_neighbors': 10}
------------------Best Params for RF----------------------
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}
------------------Best Params for xgboost----------------------
{'subsample': 1.0, 'reg_lambda': 5, 'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 12, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.8}


In [33]:
## Retraining the models with best parameters

models = {
    "Random Forest Regressor": RandomForestRegressor(
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        max_features=8, 
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1
    ),
    
    "XgboostRegressor": XGBRegressor(
        subsample = 0.7,
        reg_lambda = 2,
        reg_alpha = 0.1,
        n_estimators=200,
       max_depth=5,
       learning_rate = 0.1,
       gamma = 0.3,
       colsample_bytree = 0.7
    ),
    # "Adaboost":AdaBoostRegressor(n_estimators=50, loss = 'linear' ),
}

for name, model in models.items():

    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(
        y_train, y_train_pred
    )
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(
        y_test, y_test_pred
    )

    print(name)
    print("---------Model performance for Training set-------------")
    print(f"R2  : {model_train_r2:.4f}")

    print("---------Model performance for Test set-------------")
    print(f"R2  : {model_test_r2:.4f}")
    print("=" * 40, "\n")


Random Forest Regressor
---------Model performance for Training set-------------
R2  : 0.9723
---------Model performance for Test set-------------
R2  : 0.9325

XgboostRegressor
---------Model performance for Training set-------------
R2  : 0.9769
---------Model performance for Test set-------------
R2  : 0.8621

